### Trying to understand what happens when you shift center frequency more.

In [ ]:
import argparse
import signal
import subprocess
import sys
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import time
from helpersFunctions import *
import ipywidgets as widgets
from IPython.display import display, clear_output

sys.path.append('..')

In [ ]:
# print out the current path

from shimTool.Tool import Tool, ShimMode
from shimTool.dicomUtils import *
from shimTool.shimCompute import *


def load_tool():
    if "tool" in globals():
        del globals()["tool"]
    config = load_config("/home/heartvista/Documents/robert/ge3t_shim_tool/config.json")
    return Tool(config, debugging=True)


In [ ]:
# want to now set an ROI, or lets say look at values only in the 2nd fifth of the image, or also for the whole volume as well
def printStats(fieldmap):
    std = np.nanstd(fieldmap)
    mean = np.nanmean(fieldmap)
    print("Mean: ", mean)
    print("Std Dev: ", std)

def compare(fieldmap1, fieldmap2):
    diff = fieldmap2 - fieldmap1
    # get std dev and mean
    print("Difference Statistics:")
    printStats(diff)
    return diff


# Testing that center frequency knob moves off resonance as expected in the image

In [ ]:
# create a new shim tool object
tool = load_tool()

In [ ]:
# want to get a background
tool.doCalibrationScan()
#tool.exsiInstance.bedPosition = 0
tool.doFieldmapScan()
background = computeFieldmapFromLatestFieldmapScan(tool)


In [ ]:
print("Background Fieldmap Stats: ", printStats(background))

### Test some one off CF changes

In [ ]:
# want to change the center frequency
tool.setCenterFrequency(99)

In [ ]:
# want to acquire another field map
tool.doFieldmapScan()
actual = computeFieldmapFromLatestFieldmapScan(tool)
print("Actual Fieldmap Stats: ", printStats(actual))


In [ ]:
# whole volume:
print("Difference between my 2 scan fieldmap background and after change in center frequency:")
diff = compare(background, actual)

In [ ]:
# want to change the center frequency
tool.setCenterFrequency(-20)

In [ ]:
# want to acquire another field map
tool.doFieldmapScan()
actual = computeFieldmapFromLatestFieldmapScan(tool)
print("Actual Fieldmap Stats: ", printStats(actual))


In [ ]:
# whole volume:
print("Difference between my 2 scan fieldmap background and after change in center frequency:")
diff = compare(background, actual)

## Acquire data to show that aggregate center frequency adjustment is reflected in my fieldmaps

In [ ]:
cf_changes = list(range(-300, 301,20))
actual_changes = []
for change in cf_changes:
    print("----------------------------------------------------")
    print(f"Changing center frequency by {change}")
    tool.setCenterFrequency(change)
    tool.doFieldmapScan()
    actual = computeFieldmapFromLatestFieldmapScan(tool)
    print(f"Actual Fieldmap Stats after changing center frequency by {change}: ")
    printStats(actual)
    diff = compare(background, actual)
    actual_changes.append(np.nanmean(diff))

### Note: with a 3.5ms TE, we will see wrapping at ~|140| hz offset from cf 

In [ ]:
print("Actual Changes: ", actual_changes)
print("expected Frequency Changes: ", cf_changes)
actual_changes = np.array(actual_changes)
cf_changes = np.array(cf_changes)

difference =  cf_changes - actual_changes
print(f"From the difference of expected - achieved changes in center frequency,\n\t absolute error has a mean of {np.mean(difference)} and std dev of: ", np.std(difference))
# plot scatter of the actual changes and the expected changes
plt.figure(figsize=(15,10))
plt.plot(cf_changes, difference)
# make the plot extra wide and show every single x point
plt.xticks(cf_changes, rotation=90)
plt.yticks(range(-300,301,20))
plt.grid()
plt.title("Error of measured mean offresonance after CF changes")
plt.xlabel("Applied Center Frequency Adjustment (Hz)")
plt.ylabel("Error between desired and observed mean offresonance (Hz)")


In [ ]:
mask = np.abs(cf_changes) < 140

subset_cf_changes = cf_changes[mask]
subset_difference = difference[mask]

plt.figure(figsize=(15,10))
plt.plot(subset_cf_changes, subset_difference)
# make the plot extra wide and show every single x point
plt.xticks(subset_cf_changes, rotation=90)
plt.yticks(range(-20,21,2))
plt.grid()
plt.title("Error of measured mean offresonance after CF changes")
plt.xlabel("Applied Center Frequency Adjustment (Hz)")
plt.ylabel("Error between desired and observed mean offresonance (Hz)")

print(f"From the difference of expected - achieved changes in center frequency,\n\t absolute error has a mean of {np.mean(subset_difference)} and std dev of: ", np.std(subset_difference))

In [ ]:
mask = cf_changes < -140

subset_cf_changes = cf_changes[mask]
subset_difference = difference[mask]

plt.plot(subset_cf_changes, subset_difference)
# make the plot extra wide and show every single x point
plt.xticks(subset_cf_changes, rotation=90)
plt.yticks(range(-290,-260,3))
plt.grid()
plt.title("Error of measured mean offresonance after CF changes")
plt.xlabel("Applied Center Frequency Adjustment (Hz)")
plt.ylabel("Error between desired and observed mean offresonance (Hz)")

print(f"From the difference of expected - achieved changes in center frequency,\n\t absolute error has a mean of {np.mean(subset_difference)} and std dev of: ", np.std(subset_difference))

In [ ]:
mask = cf_changes > 160

subset_cf_changes = cf_changes[mask]
subset_difference = difference[mask]

plt.plot(subset_cf_changes, subset_difference)
# make the plot extra wide and show every single x point
plt.xticks(subset_cf_changes, rotation=90)
plt.yticks(range(270,300,3))
plt.grid()
plt.title("Error of measured mean offresonance after CF changes")
plt.xlabel("Applied Center Frequency Adjustment (Hz)")
plt.ylabel("Error between desired and observed mean offresonance (Hz)")

print(f"From the difference of expected - achieved changes in center frequency,\n\t absolute error has a mean of {np.mean(subset_difference)} and std dev of: ", np.std(subset_difference))

## get some more fine grain data within the non-wrapping range of cf adjustments

In [ ]:
tool.autoPrescanDone = False
tool.doFieldmapScan()
background = computeFieldmapFromLatestFieldmapScan(tool)

In [ ]:
print("Background Fieldmap Stats: ", printStats(background))
background_center = np.nanmean(background).astype(np.int32)
print(background_center)

In [ ]:

cf_changes = list(range(-150-background_center, 151-background_center,10))
actual_changes = []
for change in cf_changes:
    print("----------------------------------------------------")
    print(f"Changing center frequency by {change}")
    tool.setCenterFrequency(change)
    tool.doFieldmapScan()
    actual = computeFieldmapFromLatestFieldmapScan(tool)
    print(f"Actual Fieldmap Stats after changing center frequency by {change}: ")
    printStats(actual)
    diff = compare(background, actual)
    actual_changes.append(np.nanmean(diff))
actual_changes = np.array(actual_changes)
cf_changes = np.array(cf_changes)
print("Actual Changes: ", actual_changes)
print("expected Frequency Changes: ", cf_changes)
difference =  cf_changes - actual_changes
pint("differences: ", difference)
print(f"From the difference of expected - achieved changes in center frequency,\n\t absolute error has a mean of {np.mean(difference)} and std dev of: ", np.std(difference))
# plot scatter of the actual changes and the expected changes
plt.plot(cf_changes, difference)
plt.title("Error of measured mean offresonance after CF changes")
plt.xlabel("Applied Center Frequency Adjustment (Hz)")
plt.ylabel("Error between desired and observed mean offresonance (Hz)")



### get two scans done with ideal and see if their differences are similar to the differences from my scan...

In [ ]:
# acquire idea fieldmap manually. verify the ideal scans you want are here
tool.transferScanData()

for subdir in listSubDirs(tool.localExamRootDir):
    p = os.path.join(tool.localExamRootDir, subdir)
    d = listDicomFiles(p)[0]
    print(d)
    ds = pydicom.dcmread(d)
    print(getattr(ds, 'SeriesDescription'))

In [ ]:
# for each image in the directory, mask it and construct the fieldmap obtained by IDEAL sequence of the background
def getDicomArrayFromDir(dicomDir) -> np.ndarray:
    dicomArray = []
    for img in listDicomFiles(dicomDir):
        ds = pydicom.dcmread(img)
        dicomArray.append(ds.pixel_array)
    return np.stack(dicomArray, axis=0)

def getIdealFieldmapFromDir(waterDicomDir, fieldmapDicomDir, threshold=0.5):
    water = getDicomArrayFromDir(waterDicomDir).astype(np.float32)
    fieldmap = getDicomArrayFromDir(fieldmapDicomDir).astype(np.float32)
    mask = water < np.nanmean(water) * threshold
    fieldmap[mask] = np.nan
    return fieldmap

In [ ]:
waterDicomDirIDEALBackground, FieldmapDicomDirIDEALBackground = listSubDirs(tool.localExamRootDir)[-4:-2]
print(waterDicomDirIDEALBackground, FieldmapDicomDirIDEALBackground)
background_IDEAL = getIdealFieldmapFromDir(waterDicomDirIDEALBackground, FieldmapDicomDirIDEALBackground, threshold=.4)
print("'After' Fieldmap IDEAL Stats: ", printStats(background_IDEAL))

In [ ]:
waterDicomDirIDEALAfter, FieldmapDicomDirIDEALAfter = listSubDirs(tool.localExamRootDir)[-2:]
print(waterDicomDirIDEALAfter, FieldmapDicomDirIDEALAfter)
after_IDEAL = getIdealFieldmapFromDir(waterDicomDirIDEALAfter, FieldmapDicomDirIDEALAfter, threshold=.4)
print("Background Fieldmap IDEAL Stats: ", printStats(after_IDEAL))

In [ ]:
# whole volume:
print("Difference between my 2 scan fieldmap background and after change in center frequency:")
diff = compare(background_IDEAL, after_IDEAL)

# Testing before and after CF is applied for a specific solution once gradients are applied

## Setup

In [ ]:
# from the computed solutions:
    #   for a slice on isocenter (slice 31), 
    #   slice off isocenter (slice 20), 
    #   the whole volume, 
    #   and the volume of the ROI, 
    # apply the shim values and compare the results
def applyShimAndCompare(tool: Tool, sliceIdx: int = None):
    tool.setCenterFrequency(deltaCF=0)
    print("Setting gradients only now")
    tool.setLinGradients(deltaLinGrad=tool.getSolutionsToApply(sliceIdx)[1:4])
    tool.doFieldmapScan()
    before = computeFieldmapFromLatestFieldmapScan(tool)
    print("Stats of the actual acquired fieldmap after only changing the linear gradients:\n", tool.shimStatStrsVolume[2])
    print("\nSetting Center frequency now")
    tool.setCenterFrequency(deltaCF=tool.getSolutionsToApply(sliceIdx)[0])
    tool.doFieldmapScan()
    print("Stats of the actual acquired fieldmap after only changing the linear gradients:\n", tool.shimStatStrsVolume[2])
    after = computeFieldmapFromLatestFieldmapScan(tool)
    return before, after


In [ ]:
# reset the shim tool object 
tool = load_tool()

In [ ]:
# acquire a new background
tool.doCalibrationScan()
tool.doFieldmapScan()
background = computeFieldmapFromLatestFieldmapScan(tool)

# for viewing purposes
tool.getROIBackgound()
background_mag = tool.viewData[0]

In [ ]:
# acquire basis maps of the gradients
tool.doBasisCalibrationScans()

## Volume wise

In [ ]:
tool.setShimMode(ShimMode.VOLUME)
tool.recomputeCurrentsAndView()

In [ ]:
print("principle sols", tool.principleSols)
print("solutions to apply for the volume", tool.getSolutionsToApply())
print("\nstats of the background:\n", tool.shimStatStrsVolume[0])
print("\nstats for expected:\n", tool.shimStatStrsVolume[1])



In [ ]:
# Entire Volume
beforeCFMoved, afterCFMoved = applyShimAndCompare(tool, None)
print('\n\n')
diff = compare(background, beforeCFMoved)
print("Comparing background with after CF moved")
diff = compare(background, afterCFMoved)
print("Comparing before CF moved with after CF moved")
diff = compare(beforeCFMoved, afterCFMoved)

## Slice Wise

In [ ]:
tool.setShimMode(ShimMode.SLICE)
tool.recomputeCurrentsAndView()

In [ ]:
# for a slice on isocenter
# add a slider to select the slice
sliceSlider = widgets.IntSlider(description='Slice Idx:', disabled=False)
output_widget = widgets.Output()
startEvalScan = widgets.Button(description='Start Eval Scan')
sliceSlider.max = len(tool.shimStatsPerSlice[0]) - 1  # Update slider range
sliceSlider.value = 0  # Reset to first image
sliceIdx =  0 


# Update the displayed image when the slider value changes
def on_slider_value_change(change):
    if change['new'] is not None:
        sliceIdx = change['new']

        # visualize that slice
        with output_widget:
            clear_output(wait=True)
            plt.imshow(background_mag[:,sliceIdx], cmap='gray')
            plt.show()

            print("principle sols", tool.principleSols)
            print("solutions to apply for the volume", tool.getSolutionsToApply(sliceIdx))
            print("\nstats of the background:\n", tool.shimStatStrsPerSlice[0][sliceIdx])
            print("\nstats for expected:\n", tool.shimStatStrsPerSlice[1][sliceIdx])

def init_slice_eval(b):
    with output_widget:
        print("\n-------------")
        print(f"Evaluating results on Slice {sliceSlider.value}\n")
        beforeCFMoved, afterCFMoved = applyShimAndCompare(tool, sliceSlider.value)

        print("\n\nComparing background with before CF moved")
        diff = compare(background, beforeCFMoved)
        print("Comparing background with after CF moved")
        diff = compare(background, afterCFMoved)
        print("Comparing before CF moved with after CF moved")
        diff = compare(beforeCFMoved, afterCFMoved)
sliceSlider.observe(on_slider_value_change, names='value')

startEvalScan.on_click(init_slice_eval)
# Layout the widgets
widgets_layout = widgets.VBox([sliceSlider, startEvalScan, output_widget])
display(widgets_layout)


## Volume with specific ROI

In [ ]:
# volume of the ROI
tool.setShimMode(ShimMode.VOLUME)

# somehow set an ROI that makes sense to you... it would be nice to be able to spawn the gui and select the ROI manually...that would be cool... 
# for now, lets just set a circle in the center of the image
ydim, zdim, xdim = tool.viewData[0].shape
tool.ROI.setROILimits(xdim, ydim, zdim)
tool.ROI.sizes[0] = max(1, round((tool.ROI.xdim // 2) * .35))
tool.ROI.sizes[1] = max(1, round((tool.ROI.ydim // 2) * .25))
tool.ROI.sizes[2] = max(1, round((tool.ROI.zdim // 2) * .15))
tool.ROI.centers[0] = round(tool.ROI.xdim * .5)
tool.ROI.centers[1] = round(tool.ROI.ydim * .45)
tool.ROI.centers[2] = round(tool.ROI.zdim * .75)
tool.ROI.enabled = True
tool.ROI.updated = True
tool.computeMask()
tool.recomputeCurrentsAndView()
tool.cropViewDataToFinalMask()

In [ ]:
background_cropped = np.copy(background_mag).astype(np.float32)
background_cropped[~tool.finalMask] = np.nan

In [ ]:
# for a slice on isocenter
# add a slider to select the slice
sliceSlider = widgets.IntSlider(description='Slice Idx:', disabled=False)
output_widget = widgets.Output()
sliceSlider.max = len(tool.shimStatsPerSlice[0]) - 1  # Update slider range
sliceSlider.value = 0  # Reset to first image
sliceIdx =  0 


# Update the displayed image when the slider value changes
def on_slider_value_change(change):
    if change['new'] is not None:
        sliceIdx = change['new']

        # visualize that slice
        with output_widget:
            clear_output(wait=True)
            plt.imshow(background_cropped[:,sliceIdx], cmap='gray')
            plt.show()

            print("principle sols", tool.principleSols)
            print("solutions to apply for the volume", tool.getSolutionsToApply(sliceIdx))
            print("\nstats of the background:\n", tool.getSolutionStrings(0))
            print("\nstats for expected:\n", tool.getSolutionStrings(1))

sliceSlider.observe(on_slider_value_change, names='value')

# Layout the widgets
widgets_layout = widgets.VBox([sliceSlider, output_widget])
display(widgets_layout)


In [ ]:
# for a slice on isocenter
# add a slider to select the slice
startEvalScan = widgets.Button(description='Start Eval Scan')
output_widget = widgets.Output()

def init_slice_eval(b):
    with output_widget:
        print("\n-------------")
        print(f"Evaluating results on ROI")
        beforeCFMoved, afterCFMoved = applyShimAndCompare(tool, None)

        print("\n\nComparing background with before CF moved")
        diff = compare(background, beforeCFMoved)
        print("Comparing background with after CF moved")
        diff = compare(background, afterCFMoved)
        print("Comparing before CF moved with after CF moved")
        diff = compare(beforeCFMoved, afterCFMoved)

startEvalScan.on_click(init_slice_eval)
# Layout the widgets
widgets_layout = widgets.VBox([startEvalScan, output_widget])
display(widgets_layout)


## Looking to evaluate the gradients and how they get applied one by one...

In [ ]:
def applyGradients1by1(tool: Tool, sliceIdx= None):
    tool.setCenterFrequency(deltaCF=0)
    print("Setting gradients only now")
    
    deltaLinGrad=tool.getSolutionsToApply(sliceIdx)[1:4]
    linGradSol= tool.getSolutions(sliceIdx)[1:4]

    label = ["x", "y", "z"]
    appliedGradientsResults = []
    expectedGradientResults = []
    differences = []
    for i in range(3):
        print(f"Setting {label[i]} gradient only now")
        lingrad = np.zeros(3)
        lingrad[i] = deltaLinGrad[i]
        if i == 1:
            lingrad[1] = - lingrad[1]
        tool.setLinGradients(deltaLinGrad=lingrad)
        tool.doFieldmapScan()
        appliedGradientsResults.append(computeFieldmapFromLatestFieldmapScan(tool))
        expectedGradientResults.append(tool.basisB0maps[i]*linGradSol[i] + background)
        print(f"Stats of the actual acquired fieldmap after only changing the {label[i]} linear gradients:\n", tool.shimStatStrsVolume[2])
        print(f"\n Comparing expected - applied results for gradient {label[i]}")
        differences.append(compare(expectedGradientResults[i], appliedGradientsResults[i]))
    return expectedGradientResults, appliedGradientsResults, differences

### Volume wise

In [ ]:
# for a slice on isocenter
# add a slider to select the slice
output_widget = widgets.Output()
startEvalScan = widgets.Button(description='Start Eval Scan')

def init_slice_eval(b):
    with output_widget:
        print("\n-------------")
        print(f"Evaluating results on Volume\n")
        expectedEach, appliedEach, differences = applyGradients1by1(tool, None)

sliceSlider.observe(on_slider_value_change, names='value')

startEvalScan.on_click(init_slice_eval)
# Layout the widgets
widgets_layout = widgets.VBox([startEvalScan, output_widget])
display(widgets_layout)

## Determining why the y gradient is acting funky...

In [ ]:
def plotPixelFieldStrengthsVsPosition(title, points, xlabel, ylabel, xlim, ylim):
    plt.scatter(points[:,0], points[:,1])
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xlim(xlim)
    plt.ylim(ylim)

def getFieldStrengthPoints(fieldmap, c, pixelSize, fovCenter=[0,0,0], title=None):
    """
    for one dimensional direction:
        - collapse all strengths onto one dimension
        - compute the relative distance for each pixel position using pixelSize
        - create list of points (distanceFromIsocenter, fieldStrength)
        - plot the points
        - plot a line of best fit
    """
    points = [[],[],[]]
    # i want to take every non nan pixel value in the fieldmap
    # then for each pixel, get the distance from fovCenter, and the field strength and add to points list
    def get_distances(pos):
        shape = fieldmap.shape
        ret = []
        for i in range(3):
            ret.append((pos[i] - (shape[i]/2+.5)) * pixelSize[i] + fovCenter[i])
        return ret

    maxDist=0
    for y in range(fieldmap.shape[0]):
        for z in range(fieldmap.shape[1]):
            for x in range(fieldmap.shape[2]):
                if not np.isnan(fieldmap[y,z,x]):
                    fieldStrength = fieldmap[y,z,x]
                    distances = get_distances([y,z,x])
                    maxDist = max(maxDist, np.max(np.abs(distances)))
                    for i in range(3):
                        points[i].append([distances[i], fieldStrength])

    if title is not None:
        directions = ["Y", "Z", "X"]
        absheight=0
        for i in range(3):
            points[i] = np.array(points[i])
            absheight = max(absheight, np.nanmax(np.abs(points[i][:,1])))

        i = c
        plotPixelFieldStrengthsVsPosition(
            f"{title} Gradient Field Strength (Hz) vs Distance (mm) Isocenter in {directions[i]}", 
            points[i], 
            "Distance from Isocenter (mm)", 
            "Field Strength (Hz)", 
            [-maxDist, maxDist], 
            [-absheight, absheight]
            )
        plt.show()
    return [points[2], points[0], points[1]]

points = getFieldStrengthPoints(tool.basisB0maps[1], 0, [3,3,3], title="Y basis map")
points = getFieldStrengthPoints(tool.basisB0maps[0], 2, [3,3,3], title="X basis map")
points = getFieldStrengthPoints(tool.basisB0maps[2], 1, [3,3,3], title="Z basis map")

## Need to evaluate how the y gradient basis map is being acquired

In [ ]:
for i in range(20,100, 20):
    print(f"Applying y gradient at {i} ticks")
    tool.setCenterFrequency(0)
    tool.setLinGradients([0,i,0])
    tool.doFieldmapScan()
    basisRaw = computeFieldmapFromLatestFieldmapScan(tool)
    basis = basisRaw - background
    points = getFieldStrengthPoints(basis, 0, [3,3,3], title="Y basis map with gradient at {i}")

### Stability test for ticks around 40-70

In [ ]:

for i in range(50,71,10):
    for j in range(3):
        print(f"Applying y gradient at {i} ticks")
        tool.setCenterFrequency(0)
        tool.setLinGradients([0,i,0])
        tool.doFieldmapScan()
        basisRaw = computeFieldmapFromLatestFieldmapScan(tool)
        basis = basisRaw - background
        points = getFieldStrengthPoints(basis, 0, [3,3,3], title=f"Y basis map with gradient at {i}")

### Slice wise

In [ ]:
# for a slice on isocenter
# add a slider to select the slice
sliceSlider = widgets.IntSlider(description='Slice Idx:', disabled=False)
output_widget = widgets.Output()
startEvalScan = widgets.Button(description='Start Eval Scan')
sliceSlider.max = len(tool.shimStatsPerSlice[0]) - 1  # Update slider range
sliceSlider.value = 0  # Reset to first image
sliceIdx =  0 


# Update the displayed image when the slider value changes
def on_slider_value_change(change):
    if change['new'] is not None:
        sliceIdx = change['new']

        # visualize that slice
        with output_widget:
            clear_output(wait=True)
            plt.imshow(background_mag[:,sliceIdx], cmap='gray')
            plt.show()

            print("principle sols", tool.principleSols)
            print("solutions to apply for the volume", tool.getSolutionsToApply(sliceIdx))
            print("\nstats of the background:\n", tool.shimStatStrsPerSlice[0][sliceIdx])
            print("\nstats for expected:\n", tool.shimStatStrsPerSlice[1][sliceIdx])

def init_slice_eval(b):
    with output_widget:
        print("\n-------------")
        print(f"Evaluating results on Slice {sliceSlider.value}\n")
        beforeCFMoved, afterCFMoved = applyShimAndCompare(tool, sliceSlider.value)

        print("\n\nComparing background with before CF moved")
        diff = compare(background, beforeCFMoved)
        print("Comparing background with after CF moved")
        diff = compare(background, afterCFMoved)
        print("Comparing before CF moved with after CF moved")
        diff = compare(beforeCFMoved, afterCFMoved)
sliceSlider.observe(on_slider_value_change, names='value')

startEvalScan.on_click(init_slice_eval)
# Layout the widgets
widgets_layout = widgets.VBox([sliceSlider, startEvalScan, output_widget])
display(widgets_layout)